In [1]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os, math

In [129]:
def preprocess_perspective(file_name):
    ## contours를 찾는 작업
    tresh_min = 100
    tresh_max = 255

    image = cv2.imread(file_name)
    im_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh, im_bw = cv2.threshold(im_bw, tresh_min, tresh_max, 0)
    contours, hierarchy = cv2.findContours(im_bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # 모서리를 찾는 작업
    rects = [cv2.minAreaRect(each) for each in contours]
    rects = [(a,b,c) for a, b, c  in rects if ((b[0]*b[1]>9000))] # 값 9000은 추후 수정 필요
    if len(rects)>1:
        return "too many cards" # 사각형을 여러개 찾았으니 error 표시
    
    box = cv2.boxPoints(rect)
    
    box = box[np.argsort(box[:, 1])]
    if box[0][0]>box[1][0]:
        temp = box[0].copy()
        box[0] = box[1]
        box[1] = temp
    if box[2][0]<box[3][0]:
        temp = box[2].copy()
        box[2] = box[3]
        box[3] = temp
    # wrapPerspective 하는 작업
    maxWidth = 100
    maxHeight = 120
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(box,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight),borderValue = (255,255,255)) # warp에 이미지 담김
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)
    
    # wrap 이미지랑 꼭지점 담긴 행렬 돌려줌
    return warp #, temp_rect

In [160]:
tresh_min = 150
tresh_max = 255

In [161]:
warp = preprocess_perspective('../big_au/sample2/C10_0_5434.jpg')

In [162]:
warp

array([[ 76,  92,  89, ...,  83, 101,  53],
       [134, 189, 212, ..., 193, 200, 108],
       [103, 196, 237, ..., 237, 203,  86],
       ...,
       [ 86, 204, 248, ..., 255, 255, 255],
       [ 94, 204, 214, ..., 255, 255, 255],
       [ 94, 178, 162, ..., 255, 255, 255]], dtype=uint8)

흑백이미지로 바꿈

In [163]:
thresh, im_bw_warp = cv2.threshold(warp, tresh_min, tresh_max, 0)

원래 카드랑 비교해보기

In [164]:
image = cv2.imread('./test/origin_card/C10.jpg')
im_bw_origin = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin = cv2.threshold(im_bw_origin, tresh_min, tresh_max, 0)

In [165]:
im_bw_warp

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0, 255, 255, ..., 255, 255,   0],
       [  0, 255, 255, ..., 255, 255,   0],
       ...,
       [  0, 255, 255, ..., 255, 255, 255],
       [  0, 255, 255, ..., 255, 255, 255],
       [  0, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [166]:
im_bw_origin

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [  0, 255, 255, ..., 255, 255,   0],
       ...,
       [  0, 255, 255, ..., 255, 255,   0],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

해밍턴이용하기

In [167]:
(im_bw_warp==im_bw_origin).sum()

11385

원래 c10인데 다른 거랑 비교한 게 더 낮게 나오는걸 보아 이건 일단 성능은 안좋을것같..

In [168]:
image = cv2.imread('./test/origin_card/C2.jpg')
im_bw_origin_2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin_2 = cv2.threshold(im_bw_origin_2, tresh_min, tresh_max, 0)
(im_bw_warp==im_bw_origin_2).sum()

8022

In [169]:
image = cv2.imread('./test/origin_card/C3.jpg')
im_bw_origin_2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin_2 = cv2.threshold(im_bw_origin_2, tresh_min, tresh_max, 0)
(im_bw_warp==im_bw_origin_2).sum()

7337

In [170]:
image = cv2.imread('./test/origin_card/C4.jpg')
im_bw_origin_2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin_2 = cv2.threshold(im_bw_origin_2, tresh_min, tresh_max, 0)
(im_bw_warp==im_bw_origin_2).sum()

7310

In [171]:
image = cv2.imread('./test/origin_card/CA.jpg')
im_bw_origin_2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin_2 = cv2.threshold(im_bw_origin_2, tresh_min, tresh_max, 0)
(im_bw_warp==im_bw_origin_2).sum()

7682

In [172]:
image = cv2.imread('./test/origin_card/S10.jpg')
im_bw_origin_2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin_2 = cv2.threshold(im_bw_origin_2, tresh_min, tresh_max, 0)
(im_bw_warp==im_bw_origin_2).sum()

10268

In [173]:
image = cv2.imread('./test/origin_card/D10.jpg')
im_bw_origin_2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin_2 = cv2.threshold(im_bw_origin_2, tresh_min, tresh_max, 0)
(im_bw_warp==im_bw_origin_2).sum()

9497

In [174]:
image = cv2.imread('./test/origin_card/H10.jpg')
im_bw_origin_2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh1,im_bw_origin_2 = cv2.threshold(im_bw_origin_2, tresh_min, tresh_max, 0)
(im_bw_warp==im_bw_origin_2).sum()

8932